## Function to validate bacteria in both LSPN downloaded list and LPSN website (not validly published bacteria) through webscrapping

Validate species names which is in abstract: extract_ab(text) function

outputs from LLM can be validated using: validate_species_name() function

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
dblist = myclient.list_database_names()
mydb = myclient["pathogens_filtered"]
mycol = mydb["pathogens_filtered_corrected"]

lspn = mydb["lspn_final_new"]

#mydb = myclient["pathogens_filtered"]
synonym_db = mydb["new_synonyms_new"]

with open("all_bacterias_de.txt") as file:
    allspecies_temp=file.readlines()
    
allspecies_de=[re.sub("[^a-zA-Z ]","",i).lower() for i in allspecies_temp]

def find_species(name):
    URL="""https://lpsn.dsmz.de/species/"""
    URL+="-".join(name.split(" "))
    #print(name)

    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")
    species_elements = soup.find_all("title")
    for species_element in species_elements:
            species_name = species_element.text.replace("Species: ","").strip().lower()
            return species_name

        
def validate_species_name(x, ws): 
    #print(x)
    x=x.lower()
    ignore_words=["it","et","isolates","of","species", "analyses", "data","journal","with","and","to","strain","on","his","her", "spp" "species","the", "is","were","are","we","this", "he", "she", "sample", "in", "was","from"]
    if x.split(" ")[0] in ignore_words or x.split(" ")[1] in ignore_words:
        return 0
    count=lspn.count_documents({"bacteria name":x} )
    #print(count)
    if count>0:
        return x
    else:
        count=lspn.count_documents({"bacteria synonym":x} )
        #print(count)
        if count>0:
            return x
    # below uses webscrapping to validate names of non-validly published species.
    if ws:
        names=find_species(x)
        if names==x:
           #print("ws")
           return x
    
    return 0

def extract_func(text,ws, pattern_number):
    bacteria_found=[]
    # Regular expression pattern to match bacterial species
    #pattern_1 = r'\b[A-Z][a-z]+ [a-z]+\b'
    pattern_1 = r'\b[A-Z][a-z]+ [A-Za-z]+\b' # pattern for bacteria names Abc Abc / Abc abc
    pattern_2 = r'\b[A-Z][a-z]+ [(][A-Za-z]+[)]+ [A-Za-z]+\b' # pattern for bacteria names Abc (abc) Abc 
    pattern_3= r'\b[A-Z][.] [A-Za-z]+\b' # pattern for bacteria names with A. Abc / A. abc
    #query="et journal and of in to with on an" # ignore other pattern like Name et. al. 
    text=re.sub("[^a-zA-Z0-9(). ""]"," ",text)
    #print(text)
    if pattern_number==1:
        
        bacterial_species = re.findall(pattern_1, text)
        #print(bacterial_species)
        for j in set(bacterial_species):
            #print(j)  
            x=validate_species_name(j.lower(),ws)
            if x:
                bacteria_found.append(x)
        
    if pattern_number==2:
        new=set(re.findall(pattern_2, text))
        #print(len(new))
        if len(new)>0:
            bacterial_species= [re.sub('[(][A-Za-z]+[)] ',"",i) for i in new]
            #print(bacterial_species)
            for j in bacterial_species:
                #print(j) 
                x=validate_species_name(j.lower(),ws)
                if x:
                    bacteria_found.append(x)                
                        
    if pattern_number==3:   
        bacterial_species = set(re.findall(pattern_3, text))
        #print(bacterial_species)
        for j in bacterial_species:
            g_n= j.split(". ")[0].lower()
            s_n=j.split(". ")[1].lower()
            pattern_for_match="[^a-z]"+ g_n+ "[a-z]+ "+s_n+ "[^a-z]"
            #print(pattern_for_match)
            #b_name_j=re.findall(pattern_for_match,str(allspecies_de))
            #print("^"+g_n+"* "+s_n)
            result=lspn.find({ "bacteria name":{"$regex" : "^"+g_n+".* "+s_n+"/" } })

            #print(result)
            for m in result:
                #print(m)
                bacteria_found.append(re.sub('[^a-z ]',"", m['bacteria name']))   
                
            result=synonym_db.find({ "bacteria synonym":{"$regex" : "^"+g_n+".* "+s_n+"/"} })

            #print(result)
            for m in result:
                #print(m)
                bacteria_found.append(re.sub('[^a-z ]',"", m['bacteria synonym']))   
            
                
    return bacteria_found

#use ws=True if we need non-validly publised species as they are not found in LSPN downloaded CSV file.
def extract_ab(text,ws):
    text=str(text)
    bacteria_found=[]
    [bacteria_found.append(i) for i in extract_func(text,ws, pattern_number=1)] 
    [bacteria_found.append(i) for i in extract_func(text,ws, pattern_number=2)] 
    [bacteria_found.append(i) for i in extract_func(text,ws, pattern_number=3)] 
    return list(set(bacteria_found))

In [4]:
extract_ab(" ansba asidha;udh akjdb;aiuda  Corynebacterium dentalis A. utsteinense sdfsdg",ws=False)

['abditibacterium utsteinense']